In [1]:
pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pdfplumber
import pandas as pd
import os
import json

In [3]:
data_dir = "./raw_data/grid_scenarios/"

In [4]:
results = {}

In [5]:
pdf = pdfplumber.open(data_dir + "Szenariorahmen_2037_Genehmigung.pdf")

In [6]:
def clean_header1(some_df):
    some_df = some_df.iloc[:, [colnum for colnum, col in enumerate(some_df.iloc[0]) if col != ""]]
    col_header = [col.replace("\n", " ") for col in some_df.iloc[0]]
    some_df.columns = col_header
    return some_df

def clean_header(some_df):
    some_df = some_df.iloc[:, [colnum for colnum, col in enumerate(some_df.columns) if col != ""]]
    col_header = [col.replace("\n", " ") for col in some_df.columns]
    some_df.columns = col_header
    return some_df

def convert_to_float(some_df):
    for col in some_df.columns:
        some_df[col] = some_df[col].apply(lambda x: x.replace(",", ".").replace(">","").replace("*", "").replace("<",""))
    return some_df.astype(float)


### Tabelle 0

In [7]:
table=pdf.pages[3].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df

,,Installierte Leistung [GW],None,None,None,None,None,None,None,None,None,None,None,None,None,None,
0,,Energieträger,,Referenz\n2020*/2021,,Szenario\nA 2037,,Szenario\nB 2037,,Szenario\nC 2037,,Szenario\nA 2045,,Szenario\nB 2045,,Szenario\nC 2045,
1,,Kernenergie,,"4,1",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",
2,,Braunkohle,,"18,9",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",
3,,Steinkohle,,"19,0",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",
4,,Gaskraftwerke\n(zzgl. endogenem Zubau),,"32,1",,"> 38,4",,"> 38,4",,"> 38,4",,"> 34,6",,"> 34,6",,"> 34,6",
5,,Öl,,"4,7",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",,"0,0",
6,,Pumpspeicher,,"9,8",,"11,1",,"11,1",,"11,1",,"11,1",,"11,1",,"11,1",
7,,sonstige konv. Erzeugung,,"4,3",,"1,0",,"1,0",,"1,0",,"1,0",,"1,0",,"1,0",
8,,Summe konventionelle\nErzeugung,,"92,9",,"> 50,5",,"> 50,5",,"> 50,5",,"> 46,7",,"> 46,7",,"> 46,7",
9,,Wind Onshore,,"56,1",,"158,2",,"158,2",,"161,6",,"160,0",,"160,0",,"180,0",


In [8]:
df = df.iloc[:, [colnum for colnum, col in enumerate(df.iloc[0]) if col != ""]]

col_header = [col.replace("\n", " ") for col in df.iloc[0]]
df.columns = col_header
df = df.rename(columns={"Energieträger": "Energy source"})

In [9]:
df_capacity = df.iloc[1:17]
df_capacity = df_capacity[~df_capacity["Energy source"].str.contains("Summe")].set_index("Energy source")
results["power_generation"] = {"all_sources": {"Installed power [GW]": convert_to_float(df_capacity)}}

In [10]:
df_energy = df.iloc[18:20]
results["energy_consumption"] = {"Net and Gross energy demand [TWh]": convert_to_float(df_energy.set_index("Energy source"))}

In [11]:
df_energy

,Energy source,Referenz 2020*/2021,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
18,Nettostromverbrauch,478*,828,891,982,999,1025,1222
19,Bruttostromverbrauch,533*,899,961,1053,1079,1106,1303


In [12]:
df_sector = convert_to_float(df.iloc[21:25, 1:].copy())
results["sector_coupling"] = {"ecars": {"Number of vehicles[mln]": df_sector.iloc[0]}}
results["sector_coupling"]["power-to-heat"] = {"Installed power[GW]": df_sector.iloc[1]}
results["sector_coupling"]["heatpumps"] = {"Number of devices in HH and CTS[mln]": df_sector.iloc[2]}
results["sector_coupling"]["electrolysis"] = {"Installed power[GW]": df_sector.iloc[3]}

In [13]:
df_sector

,Referenz 2020*/2021,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
21,1.2,25.2,31.7,31.7,34.8,37.3,37.3
22,0.8,12.6,16.1,22.0,14.9,20.4,27.0
23,1.2,14.3,14.3,14.3,16.3,16.3,16.3
24,0.1,40.0,26.0,28.0,80.0,50.0,55.0


In [14]:
df_storage = convert_to_float(df.iloc[26:, 1:].copy())
results["storage_flexibility"] = {"solar_batteries": {"Installed power[GW]": df_storage.iloc[0]}}
results["storage_flexibility"]["utiltiy_batteries"] = {"Installed power[GW]": df_storage.iloc[1]}
results["storage_flexibility"]["demand side mgmt"] = {"Installed power[GW]": df_storage.iloc[2]}

### Tabelle 1

In [15]:
table=pdf.pages[23].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df)
df = df.set_index(df.columns[0])
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Geräte Stromverbrauch [TWh],,,,,,
Haushalte,"104,0","104,0","114,3","90,4","90,4","114,3"
GHD,"103,1","103,1","116,6","96,9","96,9","116,6"


In [16]:
results["energy_consumption"]["net_electricity"] = {"Appliance consumption in HH and CTS[TWh]": convert_to_float(df)}

### Tabelle 2&3

In [17]:
table=pdf.pages[26].extract_tables()
df = pd.DataFrame(table[0][1::],columns=table[0][0])
cols = list(df.columns)
cols[1] = "Heatpump assumptions for energy consumption"
df.columns = cols
df = clean_header(df).set_index(df.columns[1]).T.fillna(method="ffill").T
df["Szenario A 2037"] = df["Szenario A und B 2037"]
df["Szenario B 2037"] = df["Szenario A und B 2037"]

df["Szenario A 2045"] = df["Szenario A und B 2045"]
df["Szenario B 2045"] = df["Szenario A und B 2045"]
df = df.drop(columns = [col for col in df.columns if "und" in col])
df

,Szenario C 2037,Szenario C 2045,Szenario A 2037,Szenario B 2037,Szenario A 2045,Szenario B 2045
Heatpump assumptions for energy consumption,,,,,,
Ø - Fläche je Wärmepumpe HH [m²],180,190,180,180,190,190
Ø - Fläche je Wärmepumpe GHD [m²],210,220,210,210,220,220
Jahresarbeitszahl (JAZ),"3,23","3,43","3,48","3,48","3,65","3,65"
Ø – Nutzwärmeenergie HH [kWh /m² p.a.]\nth,102,93,84,84,73,73
Ø – Nutzwärmeenergie GHD [kWh /m² p.a.]\nth,122,112,101,101,88,88
Ø – Energie Warmwasser [kWh /Person p.a.]\nth,120,120,120,120,120,120
Energiebedarf je Wärmepumpe HH [kWh /a]\nel,5790,5254,4443,4443,3899,3899
Energiebedarf je Wärmepumpe GHD\n[kWh /a]\nel,8036,7284,6191,6191,5403,5403


In [18]:
results["sector_coupling"]["heatpumps"]["assumptions for heatpump consumption"] = convert_to_float(df)

In [19]:
df = pd.DataFrame(table[1][1::],columns=table[1][0])
df = clean_header(df).set_index("Wärmepumpen")
df["Szenario A 2037"] = df["Szenario A und B 2037"]
df["Szenario B 2037"] = df["Szenario A und B 2037"]

df["Szenario A 2045"] = df["Szenario A und B 2045"]
df["Szenario B 2045"] = df["Szenario A und B 2045"]
df = convert_to_float(df.drop(columns = [col for col in df.columns if "und" in col]))

df.iloc[:3]

,Szenario C 2037,Szenario C 2045,Szenario A 2037,Szenario B 2037,Szenario A 2045,Szenario B 2045
Wärmepumpen,,,,,,
Anzahl HH [Mio.],11.9,13.6,11.9,11.9,13.6,13.6
Anzahl GHD [Mio.],2.4,2.7,2.4,2.4,2.7,2.7
Anzahl Gesamt [Mio.],14.3,16.3,14.3,14.3,16.3,16.3


In [20]:
results["sector_coupling"]["heatpumps"]["non-industrial number of heatpumps [mln]"] = df.iloc[:3]
results["energy_consumption"]["net_electricity"]["non-industrial energy consumption [TWh]"] = df.iloc[3:]
results["sector_coupling"]["heatpumps"]

{'Number of devices in HH and CTS[mln]': Referenz 2020*/2021     1.2
 Szenario A 2037        14.3
 Szenario B 2037        14.3
 Szenario C 2037        14.3
 Szenario A 2045        16.3
 Szenario B 2045        16.3
 Szenario C 2045        16.3
 Name: 23, dtype: float64,
 'assumptions for heatpump consumption':                                                Szenario C 2037  \
 Heatpump assumptions for energy consumption                      
 Ø - Fläche je Wärmepumpe HH [m²]                        180.00   
 Ø - Fläche je Wärmepumpe GHD [m²]                       210.00   
 Jahresarbeitszahl (JAZ)                                   3.23   
 Ø – Nutzwärmeenergie HH [kWh /m² p.a.]\nth              102.00   
 Ø – Nutzwärmeenergie GHD [kWh /m² p.a.]\nth             122.00   
 Ø – Energie Warmwasser [kWh /Person p.a.]\nth           120.00   
 Energiebedarf je Wärmepumpe HH [kWh /a]\nel            5790.00   
 Energiebedarf je Wärmepumpe GHD\n[kWh /a]\nel          8036.00   
 
                  

### Tabelle 4

In [21]:
table=pdf.pages[29].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
cols = list(df.columns)
cols[1] = "Heating type"
df.columns = cols
df = convert_to_float(clean_header(df).set_index("Heating type"))
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Heating type,,,,,,
Großwärmepumpen [GW],7.6,6.1,12.0,8.0,6.6,13.2
Elektrodenheizer [GW],5.0,10.0,10.0,6.9,13.8,13.8
Großwärmepumpen [TWh],18.2,14.6,28.7,19.1,15.9,31.8
Elektrodenheizer [TWh],4.0,8.0,8.0,5.5,11.0,11.0


In [22]:
results["sector_coupling"]["heatpumps"]["industrial installed heating power [GW]"] = df.iloc[:2]
results["energy_consumption"]["net_electricity"]["industrial heating energy consumption [TWh]"] = df.iloc[2:]

### Tabelle 5

In [23]:
header = df.columns
header

Index(['Szenario A 2037', 'Szenario B 2037', 'Szenario C 2037',
       'Szenario A 2045', 'Szenario B 2045', 'Szenario C 2045'],
      dtype='object')

In [24]:
df = pd.DataFrame([[267.5, 334.0, 352.0, 310.5, 382.9, 469.0]], columns=header)
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
0,267.5,334.0,352.0,310.5,382.9,469.0


In [25]:
results["energy_consumption"]["net_electricity"]["Energy demand in industrial sector[TWh]"] = df

### Tabelle 6

In [26]:
table=pdf.pages[34].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
cols = list(df.columns)
cols[1] = "Variable"
df.columns = cols
df = clean_header(df)
df["transport_type"] = "undefined"
df.loc[1:4, "transport_type"] = "Electric car"
df.loc[6:10, "transport_type"] = "Plug-in-hybrid"
df.loc[12:15, "transport_type"] = "Small trucks"
df.loc[17:20, "transport_type"] = "Large trucks"
df.loc[22:25, "transport_type"] = "Overheadline-hybrid-trucks"
df.loc[27, "transport_type"] = "Rail"
df.loc[29, "transport_type"] = "Bus"
df.loc[30, "transport_type"] = "All"
df = df[df.transport_type != "undefined"]
df = df.set_index(["transport_type", "Variable"])
df

Szenario A 2037  \
transport_type             Variable                                          
Electric car               Anzahl [Mio.]                              19,0   
                           Verbrauch [kWh/100km]                      17,3   
                           Fahrleistung [km/a]                       14000   
                           Verbrauch [TWh]                            46,1   
Plug-in-hybrid             Anzahl [Mio.]                               4,0   
                           Verbrauch [kWh/100km]                      16,8   
                           Fahrleistung [km/a]                       14000   
                           ...davon rein elektrisch [km/a]            7000   
                           Verbrauch [TWh]                             4,7   
Small trucks               Anzahl [Mio.]                               2,0   
                           Verbrauch [kWh/100km]                      31,6   
                           Fahrleistung [km/a]                       22000   
                           Verbrauch [TWh]                            13,9   
Large trucks               Anzahl [Mio.]                              0,15   
                           Verbrauch [kWh/100km]                      103,   
                           Fahrleistung [km/a]                      100000   
                           Verbrauch [TWh]                            15,6   
Overheadline-hybrid-trucks Anzahl [Mio.]                               0,0   
                           Verbrauch [kWh/100km]                     137,3   
                           Fahrleistung [km/a]                      100000   
                           Verbrauch [TWh]                             0,0   
Rail                       Verbrauch [TWh]                            12,0   
Bus                        Verbrauch [TWh]                             1,0   
All                        Gesamtverbrauch [TWh]                      93,4   

                                                           Szenario B 2037  \
transport_type             Variable                                          
Electric car               Anzahl [Mio.]                              25,0   
                           Verbrauch [kWh/100km]                      17,3   
                           Fahrleistung [km/a]                       14000   
                           Verbrauch [TWh]                            60,7   
Plug-in-hybrid             Anzahl [Mio.]                               4,0   
                           Verbrauch [kWh/100km]                      16,8   
                           Fahrleistung [km/a]                       14000   
                           ...davon rein elektrisch [km/a]            7000   
                           Verbrauch [TWh]                             4,7   
Small trucks               Anzahl [Mio.]                               2,4   
                           Verbrauch [kWh/100km]                      31,6   
                           Fahrleistung [km/a]                       22000   
                           Verbrauch [TWh]                            16,7   
Large trucks               Anzahl [Mio.]                               0,2   
                           Verbrauch [kWh/100km]                     103,9   
                           Fahrleistung [km/a]                      100000   
                           Verbrauch [TWh]                            20,8   
Overheadline-hybrid-trucks Anzahl [Mio.]                               0,1   
                           Verbrauch [kWh/100km]                     137,3   
                           Fahrleistung [km/a]                      100000   
                           Verbrauch [TWh]                            13,7   
Rail                       Verbrauch [TWh]                            12,5   
Bus                        Verbrauch [TWh]                             2,0   
All                        Gesamtverbrauch [TWh]                     131,1   

 

In [27]:
results["sector_coupling"]["ecars"]["general_transport_kpis_per_transport_type"] = convert_to_float(df)

### Tabelle 8

In [28]:
table=pdf.pages[41].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
clean_header(df)

,Jahr,Szenariopfad A,Szenariopfad B,Szenariopfad C
0,2037,50 %,100 %,75 %
1,2045,75 %,100 %,100%


In [29]:
new_header = list(header)
df = pd.DataFrame([[0.5, 1.0, 0.75, 0.75, 1.0, 1.0]], columns=new_header)
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
0,0.5,1.0,0.75,0.75,1.0,1.0


In [30]:
results["storage_flexibility"]["demand side mgmt"]["marketparticipation_households[%]"] = df

### Tabelle 9

In [31]:
table=pdf.pages[43].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df).set_index("Potenzial [GW]")
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Potenzial [GW],,,,,,
Industrie,"2,5","4,1","4,1","5,3","6,7","6,7"
GHD,"2,5","3,1","3,1","3,6","5,3","5,3"


In [32]:
results["storage_flexibility"]["demand side mgmt"]["potentials in industries and commercial, trade, service[GW]"] = convert_to_float(df)

### Tabelle 10

In [33]:
table=pdf.pages[44].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])

df = clean_header(df)
df["main_source"] = "undefined"
df["sub_component"] = "undefined"

df.loc[1:2, "main_source"] = "Households"
df.loc[1:2, "sub_component"] = df.loc[1:2, "[TWh]"]
df.loc[4:6, "main_source"] = "CommerceTradeService"
df.loc[4:6, "sub_component"] = df.loc[4:6, "[TWh]"]

df.loc[7, "main_source"] = "Industrial"
df.loc[7, "sub_component"] = "All"

df.loc[8, "main_source"] = "Traffic"
df.loc[8, "sub_component"] = "All"

df.loc[9, "main_source"] = "Electrolysis"
df.loc[9, "sub_component"] = "All"

df.loc[11:12, "main_source"] = "District heat"
df.loc[11:12, "sub_component"] = df.loc[11:12, "[TWh]"]

results["energy_consumption"]["net_electricity"]["energy demand by consumer[TWh]"] = convert_to_float(df[~(df.main_source == "undefined")].drop(columns="[TWh]").set_index(["main_source", "sub_component"]))

df.loc[14:16, "main_source"] = "Grid losses"
df.loc[14:16, "sub_component"] = ["Distribution", "Transmission", "Storage_Batteries"]
df.loc[17, "main_source"] = "Power plants"
df.loc[17, "sub_component"] = "Self-consumption"

results["energy_consumption"]["gross_electricity"] = {"energy demand by electricity grid [TWh]": convert_to_float(df.loc[14:17].drop(columns="[TWh]").set_index(["main_source", "sub_component"]))}

results["energy_consumption"].keys()

dict_keys(['Net and Gross energy demand [TWh]', 'net_electricity', 'gross_electricity'])

### Tabelle 11

In [34]:
table=pdf.pages[47].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df)
df = df.rename(columns={"Photovoltaik": "Solar assumptions"}).set_index("Solar assumptions").T.fillna(method="ffill").T
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Solar assumptions,,,,,,
Installierte Leistung [GW],"345,4","345,4","345,4","400,0","400,0","445,0"
Ø Bruttozubau p.a. [GW],"19,7","19,7","19,7","16,8","16,8","18,6"
Ø Nettozubau p.a.[GW],"17,9","17,9","17,9","14,2","14,2","16,1"
Volllaststunden [h/a],950,950,950,950,950,950
Ø Lebensdauer [a],25,25,25,25,25,25


In [35]:
results["power_generation"]["solar"] = {"assumptions": convert_to_float(df)}

### Tabelle 12

In [36]:
table=pdf.pages[49].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df)
df = df.rename(columns={"Wind Onshore": "Wind onshore assumptions"}).set_index("Wind onshore assumptions").T.fillna(method="ffill").T
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Wind onshore assumptions,,,,,,
Installierte Leistung [GW],"158,2","158,2","161,6","160,0","160,0","180,0"
Ø Bruttozubau p.a. [GW],"8,7","8,7","8,9","7,3","7,3","8,1"
Ø Nettozubau p.a.[GW],"6,4","6,4","6,6","4,3","4,3","5,2"
Volllaststunden [h/a],2400,2400,2400,2500,2500,2500
Ø Lebensdauer [a],22,22,22,22,22,22


In [37]:
results["power_generation"]["wind onshore"] = {"assumptions": convert_to_float(df)}

### Tabelle 13

In [38]:
table=pdf.pages[53].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df)
df = df.rename(columns={"Gebiet [GW]": "Offshore region[GW]"}).set_index("Offshore region[GW]").iloc[:-1]
df["Szenario A 2045"] = df["Szenarien 2045"]
df["Szenario B 2045"] = df["Szenarien 2045"]
df["Szenario C 2045"] = df["Szenarien 2045"]
df = df.drop(columns=["Szenarien 2045"])
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Offshore region[GW],,,,,,
Nordsee,"46,4","54,4","54,4","64,9","64,9","64,9"
Ostsee,"4,1","4,1","4,1","5,1","5,1","5,1"


In [39]:
results["power_generation"]["wind offshore"] = {"assumptions": convert_to_float(df)}

### Tabelle 14

In [40]:
table=pdf.pages[55].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df)
df = df.rename(columns={"Biomasse": "Biomass assumptions"}).set_index("Biomass assumptions").T.fillna(method="ffill").T
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Biomass assumptions,,,,,,
Installierte Leistung [GW],"4,5","4,5","4,5","2,0","2,0","2,0"
Ø Bruttozubau p.a. [GW],"0,1","0,1","0,1","0,1","0,1","0,1"
Ø Nettozubau p.a.[GW],"-0,3","-0,3","-0,3","-0,3","-0,3","-0,3"
Volllaststunden [h/a],3000,3000,3000,3000,3000,3000
Ø Lebensdauer [a],20,20,20,20,20,20


In [41]:
results["power_generation"]["biomass"] = {"assumptions": convert_to_float(df)}

### Tabelle 15

In [42]:
table=pdf.pages[55].extract_tables()
df = pd.DataFrame(table[1][1::],columns=table[1][0])
df = clean_header(df)
df = df.rename(columns={"Wasserkraft": "Hydro assumptions"}).set_index("Hydro assumptions").T.fillna(method="ffill").T
df.loc["Volllaststunden [h/a] Laufwasser", :] = "4400"
df.loc["Volllaststunden [h/a] Speicherwasser", :] = "2800"
df = df.drop("Volllaststunden [h/a]")
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Hydro assumptions,,,,,,
Installierte Leistung [GW],"5,3","5,3","5,3","5,3","5,3","5,3"
Volllaststunden [h/a] Laufwasser,4400,4400,4400,4400,4400,4400
Volllaststunden [h/a] Speicherwasser,2800,2800,2800,2800,2800,2800


In [43]:
results["power_generation"]["hydro"] = {"assumptions": convert_to_float(df)}

### Tabelle 16

In [44]:
table=pdf.pages[56].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df)
df = df.rename(columns={"biogener Anteil Abfallkraftwerke": "Waste (bio) assumptions"}).set_index("Waste (bio) assumptions").T.fillna(method="ffill").T
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Waste (bio) assumptions,,,,,,
Installierte Leistung [GW],"1,0","1,0","1,0","1,0","1,0","1,0"
Volllaststunden [h/a],5000,5000,5000,5000,5000,5000


In [45]:
results["power_generation"]["waste(bio)"] = {"assumptions": convert_to_float(df)}

### Tabellen 17 and 18

In [46]:
tables=pdf.pages[57].extract_tables()
df = pd.DataFrame(tables[0][1::],columns=tables[0][0])
df = clean_header(df)
df = df.set_index("Kapazität [GW]").iloc[:-1]
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Kapazität [GW],,,,,,
Wind Onshore,"158,2","158,2","161,6","160,0","160,0","180,0"
Wind Offshore,"50,5","58,5","58,5","70,0","70,0","70,0"
Photovoltaik,"345,4","345,4","345,4","400,0","400,0","445,0"
Biomasse,"4,5","4,5","4,5","2,0","2,0","2,0"
Wasserkraft,"5,3","5,3","5,3","5,3","5,3","5,3"
Sonstige,"1,0","1,0","1,0","1,0","1,0","1,0"


In [47]:
results["power_generation"]["renewables"] = {"capacity[GW]": convert_to_float(df)}

In [48]:
df = pd.DataFrame(tables[1][1::],columns=tables[1][0])
df = clean_header(df)
df = df.set_index("Erzeugung [TWh]").iloc[:-3]
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Erzeugung [TWh],,,,,,
Wind Onshore,"374,0","374,0","382,0","394,0","394,0","443,3"
Wind Offshore,"167,4","193,9","193,9","230,4","230,4","230,4"
Photovoltaik,"323,2","323,2","323,2","374,3","374,3","416,4"
Biomasse,"13,4","13,4","13,4","6,0","6,0","6,0"
Wasserkraft,"21,1","21,1","21,1","21,1","21,1","21,1"
Sonstige,"5,0","5,0","5,0","5,0","5,0","5,0"


In [49]:
results["power_generation"]["renewables"] = {"generation[TWh]": convert_to_float(df)}

### Tabelle 19

In [50]:
table=pdf.pages[60].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df).rename(columns={"Energieträger [GW]": "Energy source [GW]"})
df["Szenario A 2037"] = df["A/B/C 2037"]
df["Szenario B 2037"] = df["A/B/C 2037"]
df["Szenario C 2037"] = df["A/B/C 2037"]
df["Szenario A 2045"] = df["A/B/C 2045"]
df["Szenario B 2045"] = df["A/B/C 2045"]
df["Szenario C 2045"] = df["A/B/C 2045"]
df = df.drop(columns = ["A/B/C 2037", "A/B/C 2045"]).iloc[:-1].set_index("Energy source [GW]")
df

,Bestand Mai 2022,Rückbau bis 2037,Rückbau bis 2045,in Bau,in Planung,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Energy source [GW],,,,,,,,,,,
Kernenergie,"4,1","4,1","4,1","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
Braunkohle,"18,9","18,9","18,9","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
Steinkohle,"19,0","19,0","19,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
Gas\n(zzgl. endogenem Zubau),"32,1","1,4","5,2","3,5","4,1","> 38,4","> 38,4","> 38,4","> 34,6","> 34,6","> 34,6"
Mineralölprodukte,"4,7","4,7","4,7","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"
Abfall (nicht biogen),"1,0","0,0","0,0","0,0","0,0","1,0","1,0","1,0","1,0","1,0","1,0"
Pumpspeicher,"9,8","0,0","0,0","0,1","1,2","11,1","11,1","11,1","11,1","11,1","11,1"
Sonstige Konventionelle,"3,3","3,3","3,3","0,0","0,0","0,0","0,0","0,0","0,0","0,0","0,0"


In [51]:
results["power_generation"]["fossils"] = {"generation[GW]": convert_to_float(df)}

### Tabelle 20&21

In [52]:
tables=pdf.pages[65].extract_tables()
df = pd.DataFrame(tables[0][1::],columns=tables[0][0])
cols = list(df.columns)
cols[1] = "Energy source"
df.columns = cols
df = clean_header(df)
for stype in "ABC":
    df[f"Szenario {stype} 2037"] = df.iloc[:, 2]
    df[f"Szenario {stype} 2045"] = df.iloc[:, 3]
df = df.drop(columns=df.columns[1:4]).set_index("Energy source")
df = convert_to_float(df)
results["markets"] = {"CO2-prices [EUR per t]": df.iloc[0,:],
                      "Energy prices [EUR per MWh]": df.iloc[1:,:]}
results["markets"]

{'CO2-prices [EUR per t]': Szenario A 2037    160.1
 Szenario A 2045    199.5
 Szenario B 2037    160.1
 Szenario B 2045    199.5
 Szenario C 2037    160.1
 Szenario C 2045    199.5
 Name: CO -Zertifikatspreise\n2, dtype: float64,
 'Energy prices [EUR per MWh]':                Szenario A 2037  Szenario A 2045  Szenario B 2037  \
 Energy source                                                      
 Rohöl                     35.1             33.7             35.1   
 Erdgas                    19.4             19.4             19.4   
 Steinkohle                 6.9              6.7              6.9   
 Braunkohle                 6.5              6.5              6.5   
 Kernenergie                1.7              1.7              1.7   
 
                Szenario B 2045  Szenario C 2037  Szenario C 2045  
 Energy source                                                     
 Rohöl                     33.7             35.1             33.7  
 Erdgas                    19.4             19.4 

In [53]:
df = pd.DataFrame(tables[1][1::],columns=tables[1][0])
df = clean_header(df).rename(columns={"Primärenergieträger": "Energy source"}).set_index("Energy source")
df.columns = ["Factors"]
results["markets"]["emissions"] = {"energy source factors [tCO2 per MWh_th]": convert_to_float(df)}

In [54]:
results["markets"]

{'CO2-prices [EUR per t]': Szenario A 2037    160.1
 Szenario A 2045    199.5
 Szenario B 2037    160.1
 Szenario B 2045    199.5
 Szenario C 2037    160.1
 Szenario C 2045    199.5
 Name: CO -Zertifikatspreise\n2, dtype: float64,
 'Energy prices [EUR per MWh]':                Szenario A 2037  Szenario A 2045  Szenario B 2037  \
 Energy source                                                      
 Rohöl                     35.1             33.7             35.1   
 Erdgas                    19.4             19.4             19.4   
 Steinkohle                 6.9              6.7              6.9   
 Braunkohle                 6.5              6.5              6.5   
 Kernenergie                1.7              1.7              1.7   
 
                Szenario B 2045  Szenario C 2037  Szenario C 2045  
 Energy source                                                     
 Rohöl                     33.7             35.1             33.7  
 Erdgas                    19.4             19.4 

### Tabelle 22

In [55]:
table=pdf.pages[67].extract_table()
df = pd.DataFrame(table[1::],columns=table[0])
df = clean_header(df).rename(columns={"[GW]":"Battery type"}).set_index("Battery type")
df = convert_to_float(df).iloc[:-1]
df

,Szenario A 2037,Szenario B 2037,Szenario C 2037,Szenario A 2045,Szenario B 2045,Szenario C 2045
Battery type,,,,,,
Haushaltsnahe Speicher,67.4,67.4,67.4,97.7,97.7,113.4
Großbatteriespeicher,23.7,23.7,24.2,43.3,43.3,54.5


In [56]:
results.keys()

dict_keys(['power_generation', 'energy_consumption', 'sector_coupling', 'storage_flexibility', 'markets'])

In [57]:
results_list = []
word_dict = {}
import os

if os.path.exists(data_dir + "translated.json"):
    with open(data_dir + "translated.json", "r") as f:
        translations = json.load(f) 
translations[0] = 1
translations[21] = 1
translations[22] = 1
translations[23] = 1
translations[24] = 1
translations[25] = 1
translations[26] = 1


In [58]:
result_dir = "./cleaned_data/grid_development_scenarios/"

In [61]:
def add_result(df_info, info1, info2=None, info3=None):
    file_str = "%s" % info1
    if info2 == None:
        info2 = "empty"
        file_str += "_-"
    else:
        file_str += "_%s" % info2
    if info3 == None:
        info3 = "empty"
        file_str += "_-"
    else:
        file_str += "_%s" % info3
    tmp = df_info.copy()
    tmp["Category"] = info1
    tmp["Subcategory"] = info2
    tmp["Subsubcategory"] = info3
    if type(tmp) is pd.Series:
        tmp = pd.DataFrame(tmp).T
    tmp = tmp.set_index(["Category", "Subcategory", "Subsubcategory"], append=True)
    new_cols = []
    for col in tmp.columns:
        word_dict[col] = 1
        if translations[col] == 1:
            new_cols.append(col)
        else:
            new_cols.append(translations[col])
    tmp.columns = new_cols
    new_index = [] 
    for ind in tmp.index:
        if type(ind) == tuple:
            new_index.append([])
            for ind1 in ind:
                word_dict[ind1] = 1
                if type(ind1) is int or translations[ind1] == 1:
                    new_index[-1].append(ind1)
                else:
                    new_index[-1].append(translations[ind1])
            new_index[-1] = tuple(new_index[-1])
        else:
            word_dict[ind] = 1
            if translations[ind] != 1:
                new_index.append(translations[ind])
            else:
                new_index.append(ind)
    tmp.index = pd.MultiIndex.from_tuples(new_index, names=tmp.index.names)
    tmp.to_csv(result_dir + file_str + ".csv", sep=";")
    results_list.append(tmp)

In [62]:
for key in results.keys():
    for key1 in results[key].keys():
        if type(results[key][key1]) is dict:
            for key2 in results[key][key1].keys():
                if type(results[key][key1][key2]) is dict:
                    for key3 in results[key][key1][key2].keys():
                        print("heyho")
                        print(f"{key}\t\t{key1}\t\t{key2}\t\t{key3}")
                else:
                    print(f"{key}\t\t{key1}\t\t{key2}")
                    add_result(results[key][key1][key2], key, key1, key2)
                    
        else:
            print(f"{key}\t\t{key1}")
            add_result(results[key][key1], key, key1)

power_generation		all_sources		Installed power [GW]
power_generation		solar		assumptions
power_generation		wind onshore		assumptions
power_generation		wind offshore		assumptions
power_generation		biomass		assumptions
power_generation		hydro		assumptions
power_generation		waste(bio)		assumptions
power_generation		renewables		generation[TWh]
power_generation		fossils		generation[GW]
energy_consumption		Net and Gross energy demand [TWh]
energy_consumption		net_electricity		Appliance consumption in HH and CTS[TWh]
energy_consumption		net_electricity		non-industrial energy consumption [TWh]
energy_consumption		net_electricity		industrial heating energy consumption [TWh]
energy_consumption		net_electricity		Energy demand in industrial sector[TWh]
energy_consumption		net_electricity		energy demand by consumer[TWh]
energy_consumption		gross_electricity		energy demand by electricity grid [TWh]
sector_coupling		ecars		Number of vehicles[mln]
sector_coupling		ecars		general_transport_kpis_per_tra

In [63]:
import json
with open(data_dir + "untranslated.json", "w") as f:
    json.dump(word_dict, f)

### Create few more structured excels instead of many simple CSVs

pd.concat(results_list)

results_dense = pd.concat(results_list).swaplevel(0, 1).swaplevel(1, 2)
results_dense.index.names = ["Area", "Subarea", "Category", "Subcategory"]
results_dense.loc["energy_consumption"]

results_dense.loc[("power_generation", "all_sources", "Installed power [GW]")].dropna(axis=1).to_csv("power_generation_installed_power_GW.csv")

results_dense.loc[("power_generation", slice(None), "assumptions")].dropna(axis=1).to_csv("power_generation_renewables_assumptions.csv")

results_dense.loc[("power_generation", "fossils", "generation[GW]")].dropna(axis=1).to_csv("power_generation_fossils_plandetails.csv")

results_dense.loc[("power_generation", "renewables", "generation[TWh]")].dropna(axis=1).to_csv("power_generation_renewables_gen_TWh.csv")

results_dense.loc[("energy_consumption", "net_electricity")].swaplevel(0, 1).dropna(axis=1).to_csv("net_electricity_demand_TWh.csv")

results_dense.loc[("energy_consumption", "Net/Gross energy demand [TWh]")].dropna(axis=1).to_csv("net_gross_electricity_demand_TWh.csv")

results["energy_consumption"]["gross_electricity"]["energy demand by electricity grid [TWh]"].to_csv("gross_electricity_demand_TWh.csv")

results_dense.loc[("sector_coupling")][~results_dense.loc[("sector_coupling")]["Referenz 2020*/2021"].isna()].dropna(axis=1)

results_dense.loc[("markets", "emissions")].dropna(axis=1).to_csv("emission_factors.csv", sep=";")

results_dense.loc[("power_generation")]